This is a temporary notebook for shap value analysis and plots.

In [1]:
import os
import sys

proj_dir = "/home/scai/PhenPred"
if not os.path.exists(proj_dir):
    proj_dir = "/Users/emanuel/Projects/PhenPred"
sys.path.extend([proj_dir])

import json
import PhenPred
import argparse
import pandas as pd
from PhenPred.vae import plot_folder
from PhenPred.vae.Hypers import Hypers
from PhenPred.vae.Train import CLinesTrain
from PhenPred.vae.DatasetDepMap23Q2 import CLinesDatasetDepMap23Q2
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

import optuna

In [3]:
study_name = "MOVE"
storage = f"sqlite:////home/scai/PhenPred/reports/vae/files/optuna_{study_name}.db"
study = optuna.load_study(study_name=study_name, storage=storage)
all_trials = study.trials_dataframe()
all_trials.tail(10)


,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_contrastive_neg_margin,params_contrastive_pos_margin,params_hidden_dims,params_latent_dim,params_learning_rate,params_optimizer_type,params_probability,params_scheduler_factor,params_view_dropout,params_view_latent_dim,params_w_contrastive,state
232,232,4.405551,2023-09-03 17:35:11.224674,2023-09-03 18:12:30.097678,0 days 00:37:18.873004,33,0.191717,0.790322,0.4,218,0.000321,adam,0.300827,0.689872,0.100454,0.031575,0.005662,COMPLETE
233,233,4.490187,2023-09-03 18:12:30.238280,2023-09-03 18:42:08.648575,0 days 00:29:38.410295,32,0.188606,0.793689,0.4,217,0.000332,adam,0.313178,0.691401,0.100058,0.031694,0.005899,COMPLETE
234,234,4.426555,2023-09-03 18:42:08.797038,2023-09-03 19:17:26.472621,0 days 00:35:17.675583,32,0.190587,0.796450,0.4,201,0.000352,adam,0.322211,0.670464,0.101378,0.032570,0.005986,COMPLETE
235,235,4.425823,2023-09-03 19:17:26.614077,2023-09-03 19:46:38.035651,0 days 00:29:11.421574,33,0.189183,0.799502,0.4,208,0.000332,adam,0.318712,0.672161,0.101325,0.032158,0.005917,COMPLETE
236,236,4.454983,2023-09-03 19:46:38.177668,2023-09-03 20:16:51.834850,0 days 00:30:13.657182,32,0.192139,0.799654,0.4,206,0.000331,adam,0.328162,0.672447,0.105852,0.032194,0.005897,COMPLETE
237,237,4.426590,2023-09-03 20:16:51.960817,2023-09-03 20:53:14.878106,0 days 00:36:22.917289,33,0.189878,0.800811,0.4,200,0.000331,adam,0.319582,0.676050,0.101111,0.034385,0.005825,COMPLETE
238,238,4.495154,2023-09-03 20:53:15.017714,2023-09-03 21:22:29.441230,0 days 00:29:14.423516,34,0.191236,0.800408,0.4,205,0.000348,adam,0.323913,0.674703,0.101025,0.033714,0.005928,COMPLETE
239,239,4.468220,2023-09-03 21:22:29.582524,2023-09-03 21:58:08.603921,0 days 00:35:39.021397,32,0.191403,0.798658,0.4,201,0.000334,adam,0.318248,0.674232,0.104747,0.034408,0.005951,COMPLETE
240,240,4.732317,2023-09-03 21:58:08.740673,2023-09-03 22:11:38.343408,0 days 00:13:29.602735,32,0.188073,0.803519,0.4,199,0.000204,adamw,0.311725,0.693053,0.103862,0.031914,0.005922,COMPLETE
241,241,NaN,2023-09-03 22:11:38.484551,NaT,NaT,32,0.193781,0.799591,0.4,193,0.000344,adam,0.325757,0.675327,0.100159,0.034823,0.005634,RUNNING


In [16]:
all_trials.head(100)

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_contrastive_neg_margin,params_contrastive_pos_margin,params_hidden_dims,params_latent_dim,params_learning_rate,params_optimizer_type,params_probability,params_scheduler_factor,params_view_dropout,params_view_latent_dim,params_w_contrastive,state
0,0,8.165279,2023-09-01 13:01:20.527637,2023-09-01 13:03:46.826106,0 days 00:02:26.298469,170,0.156472,0.795730,"0.7,0.4",187,6.857709e-07,adamw,0.578219,0.386044,0.245628,0.033290,0.004495,COMPLETE
1,1,5.031160,2023-09-01 13:03:46.978710,2023-09-01 13:15:11.512930,0 days 00:11:24.534220,224,0.271102,0.821786,"0.6, 0.3",252,5.868915e-04,adamw,0.243912,0.612422,0.371574,0.059274,0.004601,COMPLETE
2,2,8.176940,2023-09-01 13:15:11.650973,2023-09-01 13:17:29.697083,0 days 00:02:18.046110,108,0.051150,0.824246,0.3,64,1.508731e-06,adamw,0.557152,0.701948,0.407775,0.096492,0.006750,COMPLETE
3,3,5.051396,2023-09-01 13:17:29.835779,2023-09-01 13:34:53.243413,0 days 00:17:23.407634,98,0.289511,0.918957,"0.6, 0.3",110,1.399284e-04,adamw,0.319617,0.539111,0.340684,0.079008,0.004679,COMPLETE
4,4,5.279945,2023-09-01 13:34:53.383862,2023-09-01 13:52:40.707981,0 days 00:17:47.324119,42,0.151280,0.916386,0.5,58,1.583819e-04,adamw,0.480645,0.603804,0.595988,0.019805,0.007416,COMPLETE
5,5,5.356425,2023-09-01 13:52:40.853572,2023-09-01 14:09:37.988898,0 days 00:16:57.135326,156,0.181731,0.752257,"0.6, 0.3",53,6.194219e-05,adam,0.476272,0.641161,0.137151,0.013600,0.000703,COMPLETE
6,6,8.109289,2023-09-01 14:09:38.129633,2023-09-01 14:11:11.581898,0 days 00:01:33.452265,144,0.219257,0.764651,0.4,194,7.328817e-07,adamw,0.544242,0.369103,0.196396,0.050518,0.006699,COMPLETE
7,7,8.179326,2023-09-01 14:11:11.701105,2023-09-01 14:12:51.423956,0 days 00:01:39.722851,79,0.272592,0.736760,0.3,107,1.268282e-07,adam,0.221786,0.688346,0.540610,0.047931,0.002360,COMPLETE
8,8,100.000000,2023-09-01 14:12:51.582405,2023-09-01 14:13:14.873891,0 days 00:00:23.291486,178,0.246729,0.892703,0.5,108,2.903148e-03,adam,0.553944,0.724556,0.115322,0.062220,0.003097,COMPLETE
9,9,4.981661,2023-09-01 14:13:15.014548,2023-09-01 14:19:03.276758,0 days 00:05:48.262210,198,0.110234,0.865553,0.4,213,3.245845e-04,adam,0.293439,0.616774,0.290184,0.043876,0.007617,COMPLETE


In [13]:
all_trials.sort_values(by=["value", "number"], ascending=[False, True]).head(40)

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_contrastive_neg_margin,params_contrastive_pos_margin,params_hidden_dims,params_latent_dim,params_learning_rate,params_optimizer_type,params_probability,params_scheduler_factor,params_view_dropout,params_view_latent_dim,params_w_contrastive,state
8,8,100.0,2023-09-01 14:12:51.582405,2023-09-01 14:13:14.873891,0 days 00:00:23.291486,178,0.246729,0.892703,0.5,108,0.002903,adam,0.553944,0.724556,0.115322,0.062220,0.003097,COMPLETE
11,11,100.0,2023-09-01 14:26:16.775076,2023-09-01 14:27:09.273090,0 days 00:00:52.498014,232,0.293761,0.851122,0.7,246,0.001105,adam,0.298202,0.522494,0.410167,0.065927,0.008233,COMPLETE
13,13,100.0,2023-09-01 14:32:53.573668,2023-09-01 14:33:04.969539,0 days 00:00:11.395871,194,0.199747,0.869478,0.6,189,0.004670,adam,0.386989,0.473520,0.262888,0.038972,0.005788,COMPLETE
19,19,100.0,2023-09-01 15:25:40.098388,2023-09-01 15:26:09.568771,0 days 00:00:29.470383,206,0.161434,0.898210,0.6,29,0.002349,adam,0.180623,0.551131,0.162264,0.029007,0.009094,COMPLETE
24,24,100.0,2023-09-01 15:56:01.006984,2023-09-01 15:56:18.199621,0 days 00:00:17.192637,128,0.175529,0.702397,0.4,219,0.004671,adam,0.186637,0.499676,0.234921,0.010840,0.009993,COMPLETE
27,27,100.0,2023-09-01 16:15:33.818567,2023-09-01 16:15:57.795960,0 days 00:00:23.977393,150,0.103267,0.727214,0.4,122,0.002460,adam,0.226331,0.548027,0.136761,0.033892,0.007794,COMPLETE
29,29,100.0,2023-09-01 16:23:41.372250,2023-09-01 16:24:05.483864,0 days 00:00:24.111614,108,0.158336,0.722552,"0.7,0.4",89,0.004878,adam,0.155764,0.592707,0.115267,0.020272,0.007067,COMPLETE
30,30,100.0,2023-09-01 16:24:05.620941,2023-09-01 16:24:27.237117,0 days 00:00:21.616176,160,0.112411,0.741856,0.4,127,0.002327,adam,0.272309,0.564549,0.157446,0.034190,0.006087,COMPLETE
43,43,100.0,2023-09-01 18:42:11.253953,2023-09-01 18:42:36.446647,0 days 00:00:25.192694,59,0.148758,0.732930,0.3,135,0.002812,adam,0.205096,0.560102,0.122821,0.014494,0.007225,COMPLETE
45,45,100.0,2023-09-01 18:53:54.173523,2023-09-01 18:54:06.639408,0 days 00:00:12.465885,65,0.105231,0.773875,0.3,116,0.003321,adam,0.237246,0.519410,0.130816,0.037573,0.006648,COMPLETE
